In [15]:
# This is for my own setup

!ls -la /content
!git clone https://github.com/mehrdad-mirpourian/nlu_hw2.git
!ls -la /content/nlu_hw2


import os
os.chdir("/content/nlu_hw2")
print("Current directory:", os.getcwd())  # Should print /content/nlu_hw2

!ls -la

total 20
drwxr-xr-x 1 root root 4096 Mar 10 00:08 .
drwxr-xr-x 1 root root 4096 Mar 10 00:03 ..
drwxr-xr-x 4 root root 4096 Mar  6 14:29 .config
drwxr-xr-x 3 root root 4096 Mar 10 00:08 nlu_hw2
drwxr-xr-x 1 root root 4096 Mar  6 14:29 sample_data
Cloning into 'nlu_hw2'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 45 (delta 24), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 43.27 KiB | 2.88 MiB/s, done.
Resolving deltas: 100% (24/24), done.
total 380
drwxr-xr-x 4 root root   4096 Mar 10 01:13 .
drwxr-xr-x 1 root root   4096 Mar 10 00:08 ..
drwxr-xr-x 8 root root   4096 Mar 10 00:08 .git
-rw-r--r-- 1 root root 124125 Mar 10 00:08 hw2-pset.ipynb
-rw-r--r-- 1 root root 225936 Mar 10 00:08 My_Checks.ipynb
drwxr-xr-x 3 root root   4096 Mar 10 01:13 nlu_hw2
-rw-r--r-- 1 root root   4903 Mar 10 00:08 README.md
-rw-r--r-- 1 root root   1310 Mar 10 00:08 test_mo

In [16]:
! pip install datasets evaluate optuna --quiet # install datasets if it is not included in your environment

In [30]:
import torch
from collections.abc import Iterable
from datasets import load_dataset, Dataset

# Model and tokenizer from 🤗 Transformers
from transformers import AutoModelForSequenceClassification, \
    BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

import torch.nn as nn
from typing import Any

import evaluate

use_fp16 = torch.cuda.is_available()

import os
os.environ["WANDB_DISABLED"] = "true"

In [18]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model = BertForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# This code does exactly the same thing as the previous code cell
model = AutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
tokenizer = BertTokenizerFast.from_pretrained("prajjwal1/bert-tiny")

In [22]:
# Because 🤗 Transformers supports multiple deep learning libraries, you will
# need to use the keyword parameter return_tensors in order to indicate that
# you want your inputs to be returned in PyTorch format.
inputs = tokenizer(["Hello world!", "How are you?"], padding=True,
                   return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102,    0],
        [ 101, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]])}

In [23]:
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

print(outputs, end="\n\n")

# Use the dot operator to access parts of the output
print(outputs.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1629, -0.1532],
        [ 0.2339, -0.2121]]), hidden_states=None, attentions=None)

tensor([[ 0.1629, -0.1532],
        [ 0.2339, -0.2121]])


## **Problem 1d. Prepare Dataset (Code, 10 Points)**

In [24]:
# Load IMDb dataset and create validation split
imdb = load_dataset("imdb")
split = imdb["train"].train_test_split(.2, seed=3463)
imdb["train"] = split["train"]
imdb["val"] = split["test"]
del imdb["unsupervised"]

In [25]:
def preprocess_dataset(dataset: Dataset, tokenizer: BertTokenizerFast) \
        -> Dataset:
    """
    Problem 1d: Implement this function.

    Preprocesses a dataset using a Hugging Face Tokenizer and prepares
    it for use in a Hugging Face Trainer.

    :param dataset: A dataset
    :param tokenizer: A tokenizer
    :return: The dataset, prepreprocessed using the tokenizer
    """

    def tokenize_function(examples):
        """
        Tokenizes input text using Hugging Face BERT tokenizer.
        Handles multiple edge cases to ensure robust preprocessing.
        """

        # Edge Case: Handle missing or empty text
        if "text" not in examples or examples["text"] is None:
            examples["text"] = ["[EMPTY]"] * len(examples.get("label", [0] * len(examples)))

        # Edge Case: Handle empty strings or unexpected formats
        examples["text"] = [t if isinstance(t, str) and t.strip() else "[EMPTY]" for t in examples["text"]]

        # Edge Case: Handle different data structures (alternative keys)
        if not isinstance(examples["text"], list):
            examples["text"] = [str(examples["text"])]

        try:
            tokenized = tokenizer(
                examples["text"],
                padding="max_length",  # Ensures all sequences are of equal length
                truncation=True,       # Ensures long sequences are truncated
                max_length=512        # As per Appendix A.2 of BERT paper
            )
        except Exception as e:
            # Edge Case: Handle tokenization failures
            print(f"Tokenization error: {e}")
            return {
                "input_ids": [[0] * 512],  # Default empty tokens
                "token_type_ids": [[0] * 512],
                "attention_mask": [[0] * 512]
            }

        return tokenized

    # Edge Case: Handle large batch sizes by limiting batch processing
    tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=32)

    return tokenized_dataset




In [26]:
imdb["train"] = preprocess_dataset(imdb["train"], tokenizer)
imdb["val"] = preprocess_dataset(imdb["val"], tokenizer)
imdb["test"] = preprocess_dataset(imdb["test"], tokenizer)

# Visualize the preprocessed dataset
for k, v in imdb["val"][:2].items():
    print("{}:\n{}\n{}\n".format(k, type(v),
                                 [item[:20] if isinstance(item, Iterable) else
                                 item for item in v[:5]]))

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

text:
<class 'list'>
['As so many others ha', 'When converting a bo']

label:
<class 'list'>
[1, 0]

input_ids:
<class 'list'>
[[101, 2004, 2061, 2116, 2500, 2031, 2517, 1010, 2023, 2003, 1037, 6919, 4516, 1012, 2182, 2003, 1037, 2862, 1997, 1996], [101, 2043, 16401, 1037, 2338, 2000, 2143, 1010, 2009, 2003, 3227, 1037, 2204, 2801, 2000, 2562, 2012, 2560, 2070, 1997]]

token_type_ids:
<class 'list'>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask:
<class 'list'>
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]



# **Problem 2: Implement Experiment (50 Points in Total)**

**Problem 2a: Freeze Non-Bias Weights (Code, 10 Points)bold text bold text**

In [27]:
def init_model(trial: Any, model_name: str, use_bitfit: bool = False) -> \
        BertForSequenceClassification:
    """
    Problem 2a: Implement this function.

    This function should be passed to your Trainer's model_init keyword
    argument. It will be used by the Trainer to initialize a new model
    for each hyperparameter tuning trial. Your implementation of this
    function should support training with BitFit by freezing all non-
    bias parameters of the initialized model.

    :param trial: This parameter is required by the Trainer, but it will
        not be used for this problem. Please ignore it
    :param model_name: The identifier listed in the Hugging Face Model
        Hub for the pre-trained model that will be loaded
    :param use_bitfit: If True, then all parameters will be frozen other
        than bias terms
    :return: A newly initialized pre-trained Transformer classifier
    """

    # Ensure default behavior when use_bitfit=False: I added this line
    # Becasue during testing my code I got some unexpected output when
    # bitfit = False. Adding this line made my output behave correctly.
    for param in model.parameters():
        param.requires_grad = True  # Make all parameters trainable by default

    # If BitFit is enabled, freeze all non-bias parameters
    if use_bitfit:
        for name, param in model.named_parameters():
            if "bias" not in name:  # Freeze all non-bias parameters
                param.requires_grad = False

    return model

In [28]:
# The first parameter is unused; we just pass None to it
model = init_model(None, "prajjwal1/bert-tiny", use_bitfit=True)

# Check if weight matrix is frozen
print(model.bert.encoder.layer[0].attention.self.query.weight.requires_grad)

# Check if bias term is frozen
print(model.bert.encoder.layer[0].attention.self.query.bias.requires_grad)

False
True


**Problem 2b: Set Up Trainer and Tester (Code, 20 Points)**

In [31]:
def init_trainer(model_name: str, train_data: Dataset, val_data: Dataset,
                 use_bitfit: bool = False) -> Trainer:
    """
    Problem 2b: Implement this function.

    Creates a Trainer object that will be used to fine-tune a BERT-tiny
    model on the IMDb dataset. The Trainer should fulfill the criteria
    listed in the problem set.

    :param model_name: The identifier listed in the Hugging Face Model
        Hub for the pre-trained model that will be fine-tuned
    :param train_data: The training data used to fine-tune the model
    :param val_data: The validation data used for hyperparameter tuning
    :param use_bitfit: If True, then all parameters will be frozen other
        than bias terms
    :return: A Trainer used for training
    """

    # Step 1: Load model using init_model function that we developed earlier
    model = init_model(None, model_name, use_bitfit)

    # Step 2: Define a function for computing accuracy on validation
    def compute_metrics(eval_pred):
        metric = evaluate.load("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # Step 3: Define training arguments for our trainer
    training_args = TrainingArguments(
        output_dir="checkpoints",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=4,              # Train for 4 epochs (as per Problem 1c)
        save_total_limit=4,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        fp16=True
    )

    # Step 4: Create and return trainer object
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        compute_metrics=compute_metrics
    )

    return trainer

In [ ]:
# Creates a Trainer from a Hugging Face Model Hub identifier
trainer = init_trainer("prajjwal1/bert-tiny", imdb["train"], imdb["val"])

# Train using the trainer
trainer.train()

# Change this to whichever checkpoint you want to evalaute
eval_checkpoint_directory = "checkpoints/run-13/checkpoint-1252"

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: